In [1]:
import pandas as pd
import bamboolib as bam
import json

pd.set_option('display.float_format', lambda x: '%.2f' % x) # Set numeric value print format
pd.options.plotting.backend = "plotly" # Use plotly to generate visualizations

# COLS = ['video_id':str,
# 'trending_date':str,
# 'title':str,
# 'channel_title':str,
# 'category_id':int,
# 'publish_time':str,
# 'tags':str,
# 'views':int,
# 'likes':int,
# 'dislikes':int,
# 'comment_count':int,
# 'thumbnail_link':str,
# 'comments_disabled':bool,
# 'ratings_disabled':bool,
# 'video_error_or_removed':bool,
# 'description':str,
# ]



REGIONS = ["CA","DE","FR","GB","IN","JP","KR","RU","MX","US"] # TODO fix Korea if we can

# Population for each country from http://www.geoba.se/population.php?pc=world&type=028&year=2017&page=1
# useful?
POPULATIONS = [
    35623680, #CA
    80594016, #DE
    66822008, #FR
    64769452, #GB/UK
    1281935872, #IN
    126451400, #JP
    49237468, #KR
    121116960, #MX
    134547680 #RU
]

video_lists = []
raw_categories = []
categories = []
joined_data = []

for region_code in REGIONS:
    # videos.append(pd.read_csv(f"data/{region_code}videos.csv",encoding="utf-16-be",on_bad_lines='skip'))
    video_lists.append(pd.read_csv(f"data/{region_code}videos.csv",on_bad_lines='skip'))
    with open(f"data/{region_code}_category_id.json") as data_file:    
        data = json.load(data_file) 
    row_category_normalized = pd.json_normalize(data, ["items"], )
    raw_categories.append(row_category_normalized)
    print("Read files for region: " + region_code)



Read files for region: CA
Read files for region: DE
Read files for region: FR
Read files for region: GB
Read files for region: IN
Read files for region: JP
Read files for region: KR
Read files for region: RU
Read files for region: MX
Read files for region: US


In [2]:
# Make categories usuable
for i in range(len(REGIONS)):
    print("Make categories usable for: " + REGIONS[i])
    raw_category = raw_categories[i]
    ids = raw_category["id"]
    titles = raw_category["snippet.title"]
    dict = {"id":ids,"category_title":titles}
    category = pd.DataFrame(dict)
    category['id'] = category['id'].astype(str)
    categories.append(category)



Make categories usable for: CA
Make categories usable for: DE
Make categories usable for: FR
Make categories usable for: GB
Make categories usable for: IN
Make categories usable for: JP
Make categories usable for: KR
Make categories usable for: RU
Make categories usable for: MX
Make categories usable for: US


In [3]:
# Fix bad data
for i in range(len(REGIONS)):
    print("Fixing bad data for region: " + REGIONS[i])
    video_list = video_lists[i]

    #Force column dtypes
    video_list['video_id'] = video_list['video_id'].astype(str)
    video_list['trending_date'] = video_list['trending_date'].astype(str)
    video_list['title'] = video_list['title'].astype(str)
    video_list['channel_title'] = video_list['channel_title'].astype(str)
    video_list['category_id'] = video_list['category_id'].astype(str)
    video_list['publish_time'] = video_list['publish_time'].astype(str)
    video_list['tags'] = video_list['tags'].astype(str)
    #We had errors with RU views
    video_list['views'] = pd.to_numeric(video_list['views'], errors='coerce')
    video_list = video_list.dropna(subset=['views'])

    video_list['views'] = video_list['views'].astype(int)
    video_list['likes'] = video_list['likes'].astype(int)
    video_list['dislikes'] = video_list['dislikes'].astype(int)
    video_list['comment_count'] = video_list['comment_count'].astype(int)
    video_list['thumbnail_link'] = video_list['video_id'].astype(str)
    video_list['comments_disabled'] = video_list['video_id'].astype(bool)
    video_list['ratings_disabled'] = video_list['video_id'].astype(bool)
    video_list['video_error_or_removed'] = video_list['video_id'].astype(bool)
    video_list['description'] = video_list['video_id'].astype(str)


Fixing bad data for region: CA
Fixing bad data for region: DE
Fixing bad data for region: FR
Fixing bad data for region: GB
Fixing bad data for region: IN
Fixing bad data for region: JP
Fixing bad data for region: KR
Fixing bad data for region: RU
Fixing bad data for region: MX
Fixing bad data for region: US


In [4]:

#Tags are bad, let's fix that
for i in range(len(REGIONS)):
    print("Fixing tags: " + REGIONS[i])
    video_list = video_lists[i]
    fixed_tags = []
    for row in range(len(video_list.index)):
        row_raw_tags = video_list['tags'][1]
        row_tags_split = row_raw_tags.split('|')
        for i in range(len(row_tags_split)):
            row_tags_split[i] = row_tags_split[i].strip('\"') # replace quotation marks
        # Tags may have "[None]" in them
        if len(row_tags_split) == 1: 
            if not row_tags_split[0] or row_tags_split[0] == "[none]": # empy lists are falsy
                row_tags_split = []
        fixed_tags.append(row_tags_split)

    fixed_tags_tupple = {'tags': fixed_tags}
    fixed_tags_df = pd.DataFrame(fixed_tags_tupple)
    video_list['tags'] = fixed_tags_df['tags'].values

    # Fix trending dates (from string to date)
    video_list['trending_date'] = pd.to_datetime(video_list['trending_date'],format='%y.%d.%m')

    # Fix publish time (from string to date)
    video_list['publish_time'] = pd.to_datetime(video_list['publish_time'])

Fixing tags: CA
Fixing tags: DE
Fixing tags: FR
Fixing tags: GB
Fixing tags: IN
Fixing tags: JP
Fixing tags: KR
Fixing tags: RU
Fixing tags: MX
Fixing tags: US


In [5]:
# Insert more data
for i in range(len(REGIONS)):
    print("Insert more data into: " + REGIONS[i])

    #Insert count of tags
    video_list = video_lists[i]
    tag_count = []
    for row in video_list['tags']:
        tag_count.append(len(row))
    video_list.insert(len(video_list.columns),"tag_count",tag_count)

    # Insert length of description. NOTE: I've had to replace descriptions for the RU region
    description_lengths = []
    video_list['description'] = video_list['description'].fillna('') # We don't want NaN values, as they are considered float
    for row in video_list['description']:
        description_lengths.append(len(row))
    video_list.insert(len(video_list.columns),"description_length",description_lengths)

    # Add region
    region = [REGIONS[i]] * len(video_list.index)
    video_list.insert(len(video_list.columns),"region",region)


Insert more data into: CA
Insert more data into: DE
Insert more data into: FR
Insert more data into: GB
Insert more data into: IN
Insert more data into: JP
Insert more data into: KR
Insert more data into: RU
Insert more data into: MX
Insert more data into: US


In [6]:
# Join categories and videos
for i in range(len(REGIONS)):
    video_list = video_lists[i]
    category = categories[i]
    joined = video_list.join(category.set_index("id"), on="category_id")
    joined_data.append(joined)

# Merge all of them into one BIG set
full_data = pd.concat(joined_data)

In [11]:
# Describe the data
# for i in range(len(REGIONS)):
#     print(f"**====================================**")
#     print(f"**====== REGION : {REGIONS[i]} =======**")
#     print(f"**====================================**")
#     data = joined_data[i]
#     print(data.info())
#     print(data.describe())


# print(f"**====================================**")
# print(f"**================ TOTAL =============**")
# print(f"**====================================**")
# print(full_data.info())
# print(full_data.describe())

# Simply mentioning it should display GUI


In [10]:
canada = joined_data[1]
import pandas as pd; import numpy as np
# Step: Drop columns
canada_exclude_test = canada.drop(columns=['category_id', 'thumbnail_link', 'description', 'tags', 'video_id'])

canada_exclude_test

          video_id trending_date  \
0      LgVi6y5QIjM    2017-11-14   
1      Bayt7uQith4    2017-11-14   
2      1ZAPwfrtAFY    2017-11-14   
3      AHtypnRk7JE    2017-11-14   
4      ZJ9We4bjcg0    2017-11-14   
...            ...           ...   
40835  fn5WNxy-Wcw    2018-06-14   
40836  zAFv43lxqHE    2018-06-14   
40837  zSXG5I6Y2fA    2018-06-14   
40838  5d115sePmaU    2018-06-14   
40839  go-F6xvezAM    2018-06-14   

                                                   title  \
0      Sing zu Ende! | Gesangseinlagen vom Feinsten |...   
1      Kinder ferngesteuert im Kiosk! Erwachsene abzo...   
2      The Trump Presidency: Last Week Tonight with J...   
3                                    Das Fermi-Paradoxon   
4               18 SONGS mit Kelly MissesVlog (Sing-off)   
...                                                  ...   
40835  KINGDOM HEARTS III – E3 2018 Pirates of the Ca...   
40836                                     YMS: The Visit   
40837  Ungut umgeschult – G

In [18]:
full_data

import pandas as pd; import numpy as np
# Step: Drop columns
filtered_data = full_data.drop(columns=['category_id', 'thumbnail_link', 'description', 'tags', 'video_id'])

import pandas as pd; import numpy as np
# Step: Group by and aggregate
filtered_data.insert(len(filtered_data.columns),"views_per_total",filtered_data["views"] / filtered_data.groupby(['region'])["views"].transform("sum"))
filtered_data

      trending_date                                              title  \
0        2017-11-14         Eminem - Walk On Water (Audio) ft. Beyoncé   
1        2017-11-14                      PLUSH - Bad Unboxing Fan Mail   
2        2017-11-14  Racist Superman | Rudy Mancuso, King Bach & Le...   
3        2017-11-14                           I Dare You: GOING BALD!?   
4        2017-11-14        Ed Sheeran - Perfect (Official Music Video)   
...             ...                                                ...   
40944    2018-06-14                       The Cat Who Caught the Laser   
40945    2018-06-14                         True Facts : Ant Mutualism   
40946    2018-06-14  I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...   
40947    2018-06-14                How Black Panther Should Have Ended   
40948    2018-06-14  Official Call of Duty®: Black Ops 4 — Multipla...   

                  channel_title              publish_time     views    likes  \
0                    EminemVEVO